In [107]:
# imports
import pandas as p
from sklearn import decomposition
import traceback

# Import dataframe from "TREAD" notebook
%store -r
df = chosen_points.reset_index().copy()
df = df.drop('index', axis=1) #drop old index

In [108]:
# some info about the dataframe

display(df.head(3))

# current columns (aka features) of the dataset
print(df.columns)
print("Num of features: {df.columns.size}")

display(df.info())

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,VesselType,Status,Length,Width,Draft,IsClassA,Avg_Speed,EstimatedStatus,IsInWP,IsInSignificantWP,Route
0,245011000,2019-03-12 08:10:03,20.8970,-156.4710,2.4,311.0,325.0,PRINS HENDRIK,80,0,153,24,9.4,True,10.7,return,WP_28,<NA>,WP_28 - WP_42
1,245011000,2019-03-12 08:35:24,20.9492,-156.4762,10.7,334.0,333.0,PRINS HENDRIK,80,0,153,24,9.4,True,11.2,return,None,<NA>,WP_28 - WP_42
2,245011000,2019-03-12 09:03:23,21.0260,-156.5170,11.2,328.0,337.0,PRINS HENDRIK,80,0,153,24,9.4,True,11.3,return,None,<NA>,WP_28 - WP_42


Index(['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading',
       'VesselName', 'VesselType', 'Status', 'Length', 'Width', 'Draft',
       'IsClassA', 'Avg_Speed', 'EstimatedStatus', 'IsInWP',
       'IsInSignificantWP', 'Route'],
      dtype='object')
Num of features: {df.columns.size}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65520 entries, 0 to 65519
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype        
---  ------             --------------  -----        
 0   MMSI               65520 non-null  int32        
 1   BaseDateTime       65520 non-null  datetime64[s]
 2   LAT                65520 non-null  float64      
 3   LON                65520 non-null  float64      
 4   SOG                65520 non-null  float64      
 5   COG                65520 non-null  float32      
 6   Heading            65520 non-null  float32      
 7   VesselName         52356 non-null  string       
 8   VesselType         65520 non-null  int16       

None

#### General Approach
1. Filling missing data? (To be decided)
2. Normalization (some features have different scales)
3. Dimensionality reduction (if possible)
4. Time series alignment? (To be decided)
5. Clustering algorithm

### Normalization

### Dimensionality Reduction
In order to do reduce the number of features of the data source using the PCA methodology, some columns need to be converted into numerical form.
- 'BaseDateTime' from datetime64 to int64
- 'VesselName' -> column removed
- 'EstimatedStatus' from string to int (category)
- 'IsInWP' from string to int (category)
- 'IsInSignificantWP' from string to int (category)
- 'Route' -> Column manually removed (because it contains the same value)

In [109]:
df = df.drop(['VesselName', 'Route'], axis=1)

In [146]:
try:
    df['BaseDateTime'] = p.to_numeric(df['BaseDateTime'], errors='raise')
    df['EstimatedStatus'] = df['EstimatedStatus'].astype('category').cat.codes
    df['IsInWP'] = df['IsInWP'].astype('category').cat.codes
    df['IsInSignificantWP'] = df['IsInSignificantWP'].astype('category').cat.codes
    print(df.info())
except Exception:
    print(traceback.format_exc())

# search for any column with at least one NaN value
cols_with_nans = list(df.columns[df.isnull().any()])
if cols_with_nans:
    print(f"The columns {cols_with_nans} need their NaNs values fixed!)")
else:
    print('No NaNs values found')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65520 entries, 0 to 65519
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MMSI               65520 non-null  int32  
 1   BaseDateTime       65520 non-null  int64  
 2   LAT                65520 non-null  float64
 3   LON                65520 non-null  float64
 4   SOG                65520 non-null  float64
 5   COG                65520 non-null  float32
 6   Heading            65520 non-null  float32
 7   VesselType         65520 non-null  int16  
 8   Status             65520 non-null  int8   
 9   Length             65520 non-null  int16  
 10  Width              65520 non-null  int16  
 11  Draft              50663 non-null  float32
 12  IsClassA           65520 non-null  bool   
 13  Avg_Speed          65520 non-null  float64
 14  EstimatedStatus    65520 non-null  int8   
 15  IsInWP             65520 non-null  int8   
 16  IsInSignificantWP  655

In [148]:
# Replacing NaN values
df['Draft'] = df['Draft'].fillna(value=0)

# search for any column with at least one NaN value
cols_with_nans = list(df.columns[df.isnull().any()])
if cols_with_nans:
    print(f"The columns {cols_with_nans} need their NaNs values fixed!)")
else:
    print('No NaNs values found')

No NaNs values found


In [ ]:
'''# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
# pca dimensionality reduction
pca = decomposition.PCA(n_components='mle') # automatically detect the number of relevant features
#pca = decomposition.PCA(n_components=15)
reduced_data = pca.fit_transform(df)
reduced_df = p.DataFrame(reduced_data, columns=["Col_" + str(x) for x in range(0,15)])
display(reduced_df)
print(reduced_df.info())'''